# Descargar comentarios desde reddit

En este *notebook* se descargan comentarios desde reddit argentina.

### Fuente

- [reddit-api](https://www.jcchouinard.com/reddit-api-without-api-credentials/)

In [1]:
import praw
import pandas as pd
import os

TEXT_SAVE_FILE = 'docs/reddit_data.csv'
MIN_WORDS_COMMENTS = 5
MAX_WORDS_COMMENTS = 40


Realizamos la conexión a Reddit con las credenciales de la API que tengamos.

In [2]:
reddit = praw.Reddit(client_id=os.environ['REDDIT_CLIENT_ID'],
                     client_secret=os.environ['REDDIT_CLIENT_SECRET'],
                     user_agent=os.environ['REDDIT_USER_AGENT'])



# Objetivo

En este paso, extraemos los comentarios de reddit Argentina.

Cada *post* en reddit esta conformado por comentarios, y a la vez cada comentario puede tener respuestas. Nos interesa los comentarios en diferentes niveles, tanto los que pertenecen al *post* como los que son respuestas a otros comentarios.

Los comentarios en reddit pueden ser *link* o pueden ser solo textos. Filtramos solamente los comentarios que tengan textos. Para esto obtenemos cada comentario en formato *markdown* y eliminamos aquellos que comiencen con **>**.

## Estructura de datos

De cada comentario que se guarde de reddit, se obtendrá con los siguientes datos:

- *score*: es un puntaje que los usuarios le dan al comentario.
- *id*: identificador de reddit.
- *flair*: es el tipo de comentario etiquetado por reddit, por ejemplo, política, economía, humor, etc.
- *comms_num*: número de respuestas que recibió el comentaio.
- *comment_parent_id*: identificador del *post* o comentario al cual responde el comentario actual.
- *is_replay*: si es *False* significa que es un comentario de un post, en caso que sea *True* significa que es una respuesta a otro comentario.

In [3]:
topics_dict = { "score":[], "id":[], "flair":[], \
                "comms_num": [],  "body":[], "comment_parent_id":[], "is_replay": []}

for submission in reddit.subreddit('argentina').new(limit=50000):
    submission_comm = reddit.submission(id=submission.id)

    # replication of comment section of reddit post
    def replies_of(top_level_comment, link_flair_text, topics_dict):
        if len(top_level_comment.replies) == 0:
            return
        for num, comment in enumerate(top_level_comment.replies):
            stringCleanComment = depure(comment.body)
            if not stringCleanComment == '':
                #print('-' * 5, stringCleanComment)
                topics_dict["flair"].append(link_flair_text)
                topics_dict["score"].append(comment.score)
                topics_dict["id"].append(comment.id)
                topics_dict["is_replay"].append(True)
                topics_dict["comms_num"].append(len(comment.replies))
                topics_dict["comment_parent_id"].append(top_level_comment.id)
                topics_dict["body"].append(depure(comment.body))
                if len(comment.replies) == 0:
                    return
                replies_of(comment, link_flair_text, topics_dict)

    def depure(text):
        stringClean = ""
        for cline in text.splitlines( ):
            line = cline.strip()
            if not line.startswith('>') and len(line) < 1500 and not line.startswith('#####&#009') and not line.startswith('######&#009;')  and not line.startswith('[Owner]')  and not line.startswith('####&#009;') and not line == '' and not line.startswith('- - - - - -'): 
                if stringClean == '':
                    stringClean = line
                else:
                    stringClean = stringClean + '. ' + line
        countWords = len(stringClean.strip().split(" "))
        if countWords > MIN_WORDS_COMMENTS and countWords < MAX_WORDS_COMMENTS:
            return stringClean
        else:
            return ''

    for count, top_level_comment in enumerate(submission_comm.comments):
        count_comm = 0
        try :
            stringCleanComment = depure(top_level_comment.body)
            if not stringCleanComment == '':
                topics_dict["flair"].append(submission.link_flair_text)
                topics_dict["score"].append(top_level_comment.score)
                topics_dict["id"].append(top_level_comment.id)
                topics_dict["is_replay"].append(False)
                topics_dict["comms_num"].append(len(top_level_comment.replies))
                topics_dict["comment_parent_id"].append(submission.id)
                topics_dict["body"].append(stringCleanComment)
            replies_of(top_level_comment, submission.link_flair_text, topics_dict)
        except:
            continue



# Objetivo

Se guardan los datos obtenidos en el paso anterior en un archivo csv.

In [4]:
#print('score ' + str(len(topics_dict['score'])))
#print('flair ' + str(len(topics_dict['flair'])))
#print('id ' + str(len(topics_dict['id'])))
#print('is_replay ' + str(len(topics_dict['is_replay'])))
#print('comms_num ' + str(len(topics_dict['comms_num'])))
#print('comment_parent_id ' + str(len(topics_dict['comment_parent_id'])))
#print('body ' + str(len(topics_dict['body'])))

topics_data = pd.DataFrame(topics_dict)
#print('Total ' + str(len(topics_data['score'])) )

#print(topics_data.head(10))

topics_data.to_csv(TEXT_SAVE_FILE, index=False)


In [5]:
print(topics_data.head(10))

   score       id            flair  comms_num  \
0      1  hgw5p7y  Humor:snoo_joy:          0   
1      1  hgw69er  Humor:snoo_joy:          0   
2      1  hgw6zvd            Meme💩          0   
3      1  hgw24ns            Meme💩          0   
4      1  hgw38x8            Meme💩          0   
5      1  hgw2rml            Meme💩          1   
6      1  hgw3wei            Meme💩          0   
7      1  hgw6rim            Meme💩          0   
8      1  hgvyymo          Video📽️          1   
9      1  hgw40yq          Video📽️          1   

                                                body comment_parent_id  \
0                       Wado "el operador" De Pedro?            q9i4uj   
1  Este Esteban Lamothe estaba en la ficción de u...            q9i4uj   
2  Eso porque son todos útos chupa bija.. Venga e...           hgw2528   
3  mas verso burgués que Maximo no hay. Es la rep...            q9hut7   
4  Ayudar con comida? Na mejor unos afiches a tod...            q9hut7   
5             ¿P